<a href="https://colab.research.google.com/github/eTroupe5201/SE489_Group_Project/blob/main/sentiment_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install hugging face dataset, accelerate and transformers library

In [ ]:
!pip install datasets
!pip install transformers -U
!pip install accelerate -U


Import rotten tomatoes dataset from huggingface. Data is automatially split for training, validation, and testing.

*   Training split: 8530 examples
*   Validation split: 1066 examples
*   Test split: The remaining examples



In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("rotten_tomatoes", split="train")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Initialize tokenizer and model. Utilizing Distilbert model from hugging face and a pre-trained tokenizer. This model has been fine-tuned on the Stanford Sentiment Treebank dataset.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
# Initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Import pytorch library, training arguments and dataloader
#Define Training Arguments:

*   Learning rate hyperparmeter: is set to 2e-5. It controls how much we are adjusting the model parameters during training.
*   Epochs: set to 1.This is the number of training epochs. It specifies how many times the entire dataset will be passed through.
*   Batch Size: is set to 32. It determines the number of samples processed in each iteraion during training.



In [ ]:
import torch
from torch.utils.data import DataLoader

from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./output',                    # Specify the output directory
    per_device_train_batch_size=16,           # Batch size per GPU during training
    num_train_epochs=1,                       # Number of training epochs
    logging_dir='./logs',                     # Directory for logging training metrics
    logging_steps=500,                        # Interval for logging training metrics
    save_steps=1000,                          # Interval for saving model checkpoints
    evaluation_strategy="epoch",              # Evaluation strategy (perform evaluation after each epoch)
    learning_rate=2e-5,                       # Learning rate
)

# You can access the training parameters like this:
learning_rate = training_args.learning_rate
num_epochs = training_args.num_train_epochs
batch_size = training_args.per_device_train_batch_size



In [ ]:
# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)


#PREPROCESS DATA


*   Tokenize Data: tokenize entire dataset
*   Padding: set to true, ensures that sequences have the same length
* Truncating: ensures that sequences are not longer than the maximum length allowed.


Convert label data into Pytourch tensors

Create Pytorch Dataloader


In [ ]:

# Tokenize and pad the entire dataset
tokenized_dataset = tokenizer(dataset['text'], padding=True, truncation=True, return_tensors='pt')

# Convert labels to tensors
labels = torch.tensor(dataset['label'])

# Create DataLoader with aligned batch sizes
dataloader = DataLoader(list(zip(tokenized_dataset['input_ids'], tokenized_dataset['attention_mask'], labels)), batch_size=batch_size, shuffle=True)


#PREDICTIONS
It is considered good practice predict outcomes before training the model.

torch.no_grad: disables gradient calculation. This speeds up computation and reduces memory consumption.

We unpack the dictionary and compute the output.

logits: raw scores for each class in the classfication task.

torch.argmax(outputs.logits, dim=1) find the index of hte class with the highest confidence socre and specified dimension.

Predictions: contains the predicted class labels for the input samples based on the model's output logits.


In [ ]:
import torch

# Define the mapping between class indices and sentiment labels
#sentiment_mapping = {0: "Negative", 1: "Positive"}

# Initialize a list to store all predictions
all_predictions = []

# Iterate over your dataset in batches
for batch_start in range(0, len(tokenized_dataset["input_ids"]), batch_size):
    # Get the current batch
    batch_end = min(batch_start + batch_size, len(tokenized_dataset["input_ids"]))
    batch_input_ids = tokenized_dataset["input_ids"][batch_start:batch_end]
    batch_attention_mask = tokenized_dataset["attention_mask"][batch_start:batch_end]

    # Pass the tokenized text through the model
    with torch.no_grad():
        batch_outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)

    # Interpret the outputs
    batch_predictions = torch.argmax(batch_outputs.logits, dim=1)

    # Append batch predictions to the list
    all_predictions.extend(batch_predictions.tolist())

    # Print progress
    print(f"Processed batch {batch_start // batch_size + 1}/{len(tokenized_dataset['input_ids']) // batch_size}")

    # Clear variables to free up memory
    del batch_input_ids, batch_attention_mask, batch_outputs

# Convert the list of predictions to a tensor
all_predictions_tensor = torch.tensor(all_predictions)

# Save all predictions to a file
torch.save(all_predictions_tensor, 'predictions.pt')

print("All predictions saved to 'predictions.pt'")
#save tensor to file


Processed batch 1/533
Processed batch 2/533
Processed batch 3/533
Processed batch 4/533
Processed batch 5/533
Processed batch 6/533
Processed batch 7/533
Processed batch 8/533
Processed batch 9/533
Processed batch 10/533
Processed batch 11/533
Processed batch 12/533
Processed batch 13/533
Processed batch 14/533
Processed batch 15/533
Processed batch 16/533
Processed batch 17/533
Processed batch 18/533
Processed batch 19/533
Processed batch 20/533
Processed batch 21/533
Processed batch 22/533
Processed batch 23/533
Processed batch 24/533
Processed batch 25/533
Processed batch 26/533
Processed batch 27/533
Processed batch 28/533


KeyboardInterrupt: 

In [ ]:
import torch.nn as nn

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()
# Define the print frequency
print_freq = 10  # Print loss every 10 batches

# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    # Set model to training mode
    model.train()

    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for batch_idx, batch in enumerate(dataloader):
        input_ids, attention_mask, batch_labels = batch

        optimizer.zero_grad()

        # Forward pass: compute predicted outputs
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Compute loss
        loss = criterion(logits, batch_labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update statistics
        running_loss += loss.item() * input_ids.size(0)
        _, predicted = torch.max(logits, 1)
        correct_predictions += (predicted == batch_labels).sum().item()
        total_predictions += batch_labels.size(0)

        if (batch_idx + 1) % print_freq == 0:
            print(f'Batch {batch_idx + 1}/{len(dataloader)}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions / total_predictions

    print(f'Epoch {epoch + 1} completed, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')



Epoch 1/1
----------
Batch 10/534, Loss: 0.5163
Batch 20/534, Loss: 0.2637
Batch 30/534, Loss: 0.0421
Batch 40/534, Loss: 0.2683
Batch 50/534, Loss: 0.7550
Batch 60/534, Loss: 1.1101
Batch 70/534, Loss: 0.6768
Batch 80/534, Loss: 0.7672
Batch 90/534, Loss: 0.1090
Batch 100/534, Loss: 0.8359
Batch 110/534, Loss: 1.0757
Batch 120/534, Loss: 0.4553
Batch 130/534, Loss: 0.7175
Batch 140/534, Loss: 1.0529
Batch 150/534, Loss: 0.0343
Batch 160/534, Loss: 0.4166
Batch 170/534, Loss: 1.1272
Batch 180/534, Loss: 0.1699
Batch 190/534, Loss: 1.4220
Batch 200/534, Loss: 0.8417
Batch 210/534, Loss: 0.6792
Batch 220/534, Loss: 1.1909
Batch 230/534, Loss: 0.3944
Batch 240/534, Loss: 0.3961
Batch 250/534, Loss: 0.0543
Batch 260/534, Loss: 0.1474
Batch 270/534, Loss: 0.2926
Batch 280/534, Loss: 0.8253
Batch 290/534, Loss: 1.0402
Batch 300/534, Loss: 0.5623
Batch 310/534, Loss: 0.8850
Batch 320/534, Loss: 1.3953
Batch 330/534, Loss: 0.3863
Batch 340/534, Loss: 0.0924
Batch 350/534, Loss: 0.0109


In [ ]:
# Validation loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
with torch.no_grad():  # No need to compute gradients during validation
    for batch in validation_dataloader:
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)
        labels = batch['label']

        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)

        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Validation Accuracy: {accuracy}")


In [ ]:
# Test loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
with torch.no_grad():  # No need to compute gradients during testing
    for batch in test_dataloader:
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)
        labels = batch['label']

        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)

        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy}")


Write a brief project description, including:
A. The overall goal of the project.

*   To develop a machine learning model for sentiment analysis on movie reviews, with the aim of accurately classifying each review as either positive or negative. By leveraging pre-trained models and fine-tuning techniques, the overall goal is to create a reliable classifier that effectively analyzes large volumes of text data.

B. The framework you plan to use, and how you intend to incorporate it into your project.

*   PyTorch: transformers
*   HuggingFace: Model, Dataset
*   Google Colab
*   TensorFlow


C. The data you initially plan to use (this may change).

*   Hugging Face Dataset: Rotten Tomatoes

D. The models you expect to use.

*   DistilBertForSequenceClassification

